# Snorkel

## Introduction

The goal of this lab is to introduce students to the [Snorkel](http://www.snorkel.org) tool and the possibilities of programmatic label generation using the weak-supervised learning paradigm.

In order to use weakly supervised learning to generate labels, it is necessary to create three datasets:

- **train set**: which does not have any labels
- **validation set**: used for hyperparameter optimization, has labels
- **test set**: used only for final model evaluation, has labels

## Labeling functions

The first step will be to load the dataset and split it into a train set and a test set. Since in our set all SMS have a label, we will simulate a weakly supervised learning problem by randomly removing 80% of the labels. Additionally, Snorkel requires numeric labels, so we need to recode the values.

In [1]:
!head data/smsspamcollection.csv

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [2]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 600)

SPAM = 1
HAM = 0
ABSTAIN = -1

df = pd.read_csv('./data/smsspamcollection.csv', sep='\t', header=None, names=['old_label', 'text'])

df['label'] = df.old_label.apply(lambda x: SPAM if x == 'spam' else HAM)

df.loc[df.sample(frac=0.8).index, 'label'] = ABSTAIN
df.drop(columns=['old_label'], inplace=True)

df.head()

,text,label
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",-1
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,U dun say so early hor... U c already then say...,-1
4,"Nah I don't think he goes to usf, he lives around here though",-1


In [3]:
abstain_idx = df.label == ABSTAIN

df_train = df[abstain_idx]
df_test = df[~abstain_idx]

### Simple keyword search

As a first example, we will use a search for the words "check" and "free" in SMS content

In [4]:
from snorkel.labeling import labeling_function

@labeling_function()
def check(sms):
    return SPAM if "check" in sms.text.lower() else ABSTAIN

@labeling_function()
def free(sms):
    return SPAM if "free" in sms.text.lower() else ABSTAIN

The next step is to apply the labeling functions to the train set.

In [5]:
from snorkel.labeling import PandasLFApplier

lfs = [check, free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 75085.66it/s]


The result of applying the set of labeling functions to the train set is a matrix of size $m \times n$, where $m$ is the number of examples and $n$ is the number of labeling functions. The matrix contains the result of applying each function to each example.

In [6]:
L_train

array([[-1, -1],
       [-1, -1],
       [-1, -1],
       ...,
       [-1, -1],
       [-1, -1],
       [-1,  1]])

In [10]:
df_train.iloc[1,:]

text     U dun say so early hor... U c already then say...
label                                                   -1
Name: 3, dtype: object

The simplest way to analyze this is to determine the coverage of labeling functions (i.e., the percentage of cases for which the function returned a result other than `ABSTAIN'.

In [11]:
coverage_check, coverage_free = (L_train != ABSTAIN).mean(axis=0)

print(f"Coverage for check(): {coverage_check * 100:.1f}%")
print(f"Coverage for free(): {coverage_free * 100:.1f}%")

Coverage for check(): 1.2%
Coverage for free(): 4.7%


Fortunately, Snorkel offers additional tools that allow for deeper analysis of the result of labeling functions.

In [12]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011664,0.0,0.0
free,1,[1],0.046882,0.0,0.0


The meaning of each column is as follows:
- `Polarity`: the set of labels returned by the function
- `Coverage`: the percentage of examples for which the function returns a value other than `ABSTAIN`
- Overlaps: the percentage of examples for which at least one other labeling function returned a value
- Conflicts: the percentage of examples for which at least one other labeling function returned a different value

If the train set contained labels, the method would also return:
- `Correct`: the number of correct labels
- `Incorrect`: number of incorrect labels
- `Empirical Accuracy`: the percentage of correct labels

Let's check the examples labeled by the `free()` function as spam

In [13]:
df_train.iloc[L_train[:,1] == SPAM].sample(frac=0.1)

,text,label
319,December only! Had your mobile 11mths+? You are entitled to update to the latest colour camera mobile for Free! Call The Mobile Update Co FREE on 08002986906,-1
3595,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 08000930705,-1
3954,Refused a loan? Secured or Unsecured? Can't get credit? Call free now 0800 195 6669 or text back 'help' & we will!,-1
4460,"Welcome to UK-mobile-date this msg is FREE giving you free calling to 08719839835. Future mgs billed at 150p daily. To cancel send ""go stop"" to 89123",-1
1492,"Your account has been credited with 500 FREE Text Messages. To activate, just txt the word: CREDIT to No: 80488 T&Cs www.80488.biz",-1
566,"Ill call u 2mrw at ninish, with my address that icky American freek wont stop callin me 2 bad Jen k eh?",-1
5467,Get your garden ready for summer with a FREE selection of summer bulbs and seeds worth £33:50 only with The Scotsman this Saturday. To stop go2 notxt.co.uk,-1
2990,No 1 POLYPHONIC tone 4 ur mob every week! Just txt PT2 to 87575. 1st Tone FREE ! so get txtin now and tell ur friends. 150p/tone. 16 reply HL 4info,-1
3978,Great NEW Offer - DOUBLE Mins & DOUBLE Txt on best Orange tariffs AND get latest camera phones 4 FREE! Call MobileUpd8 free on 08000839402 NOW! or 2stoptxt T&Cs,-1
2897,"pdate_Now - Double mins and 1000 txts on Orange tariffs. Latest Motorola, SonyEricsson & Nokia & Bluetooth FREE! Call MobileUpd8 on 08000839402 or call2optout/!YHL",-1


It seems that the phrase "call now" is also a good indicator for spam. So let's add one more labeling function.

In [14]:
@labeling_function()
def call_now(sms):
    return SPAM if "call now" in sms.text.lower() else ABSTAIN

lfs = [check, free, call_now]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 71835.23it/s]


In [15]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011664,0.000000,0.0
free,1,[1],0.046882,0.001122,0.0
call_now,2,[1],0.004486,0.001122,0.0


Let's see which examples were labeled as spam by the `call_now()` function but omitted by `free()`.

In [16]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 1], L_train[:, 2])
buckets

{(-1, -1): array([   0,    1,    2, ..., 4454, 4455, 4456]),
 (1,
  -1): array([   3,    7,   10,   46,   73,   79,  114,  120,  139,  144,  149,
         195,  210,  212,  248,  277,  286,  298,  302,  312,  326,  384,
         389,  411,  451,  462,  473,  482,  502,  539,  566,  593,  645,
         693,  720,  760,  807,  811,  865,  919,  962,  982,  987,  989,
        1024, 1088, 1098, 1112, 1147, 1164, 1174, 1204, 1215, 1216, 1225,
        1243, 1264, 1295, 1318, 1335, 1354, 1365, 1399, 1420, 1433, 1437,
        1480, 1520, 1526, 1533, 1544, 1570, 1577, 1597, 1606, 1639, 1670,
        1692, 1702, 1722, 1733, 1756, 1789, 1822, 1837, 1853, 1855, 1893,
        1900, 1903, 1911, 1956, 1995, 2028, 2053, 2067, 2112, 2126, 2142,
        2177, 2254, 2258, 2290, 2299, 2309, 2312, 2328, 2375, 2398, 2407,
        2424, 2513, 2516, 2529, 2557, 2589, 2628, 2656, 2672, 2675, 2686,
        2712, 2732, 2759, 2762, 2774, 2780, 2823, 2831, 2847, 2897, 2901,
        2950, 2964, 2977, 2993, 3011, 30

In [19]:
df_train.iloc[buckets[(ABSTAIN, SPAM)]]

,text,label
1366,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870..k",-1
1502,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
2170,"Shop till u Drop, IS IT YOU, either 10K, 5K, £500 Cash or £100 Travel voucher, Call now, 09064011000. NTT PO Box CR01327BT fixedline Cost 150ppm mobile vary",-1
2850,"YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call",-1
2871,"YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call.",-1
2992,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
3167,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870..k",-1
3893,URGENT This is our 2nd attempt to contact U. Your £900 prize from YESTERDAY is still awaiting collection. To claim CALL NOW 09061702893. ACL03530150PM,-1
4073,Loans for any purpose even if you have Bad Credit! Tenants Welcome. Call NoWorriesLoans.com on 08717111821,-1
4108,"HOT LIVE FANTASIES call now 08707500020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1


In [20]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011664,0.000000,0.0
free,1,[1],0.046882,0.001122,0.0
call_now,2,[1],0.004486,0.001122,0.0


#### assignment

Write a labeling function that marks as spam all messages containing the word "HOT" written in capitals.

In [27]:
@labeling_function()
def hot(sms):
    return SPAM if "HOT" in sms.text else ABSTAIN

### Searching based on a regular expression

Another type of labeling function is one that uses regexp to find specific expressions.

In [31]:
import re

@labeling_function()
def regex_I_am_free(sms):
    if re.search(r"((I)|(she)|(he)|(we)|(them)|(we))\s.*\bfree", sms.text, flags=re.I):
        return HAM
    elif re.search(r"free", sms.text, flags=re.I):
        return SPAM
    else:
        return ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, hot]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 27047.89it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011664,0.000000,0.000000
free,1,[1],0.046882,0.046882,0.011664
call_now,2,[1],0.004486,0.002243,0.000449
regex_I_am_free,3,"[0, 1]",0.046882,0.046882,0.011664
hot,4,[1],0.001570,0.001122,0.000000


Let's compare examples that the `free()` function labels as spam and the `regex_I_am_free()` function considers valid.

In [32]:
buckets = get_label_buckets(L_train[:, 1], L_train[:, 3])
df_train.iloc[buckets[(SPAM, HAM)]].sample(10, random_state=1)

,text,label
3526,I not free today i haf 2 pick my parents up tonite...,-1
180,Hi! You just spoke to MANEESHA V. We'd like to know if you were satisfied with the experience. Reply Toll Free with Yes or No.,-1
4386,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 8000930705,-1
3595,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 08000930705,-1
3493,I will reach office around &lt;DECIMAL&gt; . &amp; my mobile have problem. You cann't get my voice. So call you asa i'll free,-1
5058,Hey next sun 1030 there's a basic yoga course... at bugis... We can go for that... Pilates intro next sat.... Tell me what time you r free,-1
3692,"Sir, i am waiting for your call, once free please call me.",-1
4091,We tried to call you re your reply to our sms for a video mobile 750 mins UNLIMITED TEXT + free camcorder Reply of call 08000930705 Now,-1
4949,"Hi this is Amy, we will be sending you a free phone number in a couple of days, which will give you an access to all the adult parties...",-1
2434,"Indians r poor but India is not a poor country. Says one of the swiss bank directors. He says that "" &lt;#&gt; lac crore"" of Indian money is deposited in swiss banks which can be used for 'taxless' budget for &lt;#&gt; yrs. Can give &lt;#&gt; crore jobs to all Indians. From any village to Delhi 4 lane roads. Forever free power suply to more than &lt;#&gt; social projects. Every citizen can get monthly &lt;#&gt; /- for &lt;#&gt; yrs. No need of World Bank &amp; IMF loan. Think how our money is blocked by rich politicians. We have full rights against corrupt politicians. Itna forwa...",-1


#### assignment

Write a labeling function that will mark as spam all messages containing any amounts specified with a currency symbol ($99, £1.50)

In [42]:
@labeling_function()
def contains_money(sms):
    return SPAM if re.search(r"((\$)|(\£)\d)", sms.text) else ABSTAIN

### Searching based on heuristics

A simple heuristic to find spam is to assume that if more than 10% of the message text is written in capitals, there is a good chance it is spam.

In [43]:
@labeling_function()
def has_many_uppercase_words(sms):
    percentage_uppercase = sum([word.isupper() for word in sms.text.split()]) / len(sms.text.split())
    
    return SPAM if percentage_uppercase > 0.1 else ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, contains_money]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 20819.53it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.011664,0.001795,0.000000
free,1,[1],0.046882,0.046882,0.011664
call_now,2,[1],0.004486,0.004038,0.000449
regex_I_am_free,3,"[0, 1]",0.046882,0.046882,0.011664
has_many_uppercase_words,4,[1],0.182144,0.041498,0.002243
contains_money,5,[1],0.048004,0.030058,0.000449


#### assignment

Write a labeling function that marks as valid those messages that are shorter than 10 words and do not contain any word written in capitals.

In [44]:
@labeling_function()
def short_and_no_uppercase(sms):
    pass

### Using an external statistical model

When labeling data, you can use external models whose response can be important information for deciding how to label an example. Snorkel has several built-in integrations in the form of the `Preprocessor` interface, in the example below we will use the `SpaCy` library to perform additional grammatical analysis of the text. However, you will need to download the English language model.

In [40]:
# !python -m spacy download en_core_web_sm

/home/drew/prog/venvs/ml/bin/python: No module named spacy


In [ ]:
# !python -m spacy validate

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [ ]:
_text = """I don't England is a country that is part of the United Kingdom. 
It shares land borders with Wales to its west and Scotland to its north. 
The Irish Sea lies northwest of England and the Celtic Sea to the southwest. 
England is separated from continental Europe by the North Sea to the east and the 
English Channel to the south. The country covers five-eighths of the island of 
Great Britain, which lies in the North Atlantic, and includes over 100 smaller islands, 
such as the Isles of Scilly and the Isle of Wight."""

doc = nlp(_text)

for e in doc.ents:
    print(e.text, e.label_)

In [ ]:
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

Assume that short text messages in which a reference to a specific person appears are not spam.

In [ ]:
df_train.columns

In [ ]:
@labeling_function(pre=[spacy])
def has_person(sms):
    if len(sms.doc) < 20 and any([ent.label_ == "PERSON" for ent in sms.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [ ]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

Another example of pre-processing data for labeling would be determining the average word frequency of a document. Below we define a function that determines the average word frequency and we decorate it as an example of a pre-processor. When a text message is sent to the next labeling function, the pre-processor will populate the text message with the average word frequency and, based on that, the labeling function will make a decision (we assume that if the text message contains many rare words then it is spam).

In [ ]:
from wordfreq import zipf_frequency
from snorkel.preprocess import preprocessor

@preprocessor(memoize=True)
def avg_word_freq(sms):
    sms.avg_word_freq = sum([zipf_frequency(word, 'en') for word in sms.text.split()]) / len(sms.text.split())
    
    return sms

In [ ]:
@labeling_function(pre=[avg_word_freq])
def many_rare_words(sms):
    return ABSTAIN if sms.avg_word_freq >= 4 else SPAM

In [ ]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
df_train.iloc[L_train[:,6] == SPAM].sample(frac=0.1)

#### assignment

Write a labeling function that marks messages containing more than 3 adjectives as spam. Use the SpaCy library for pre-processing. 

__Hint__: the following example shows how to read the part-of-speech label for each token from the message being analyzed. For information on all token properties recognized by SpaCy, see [API documentation](https://spacy.io/api/token)

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

sms = "Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping."

for token in nlp(sms):
    print(f"{token.text:<10} {token.pos_:<10} {token.tag_:<10} {token.lemma_:<10}")

## Combining labeling functions into a single model

The goal of labeling functions is not to achieve individually large coverage. Labeling functions are inherently noisy and can make many individual errors. The true utility of labeling functions becomes apparent when multiple functions are combined to form a single model.

We will first build a simple model based on majority voting, and then build a more complex model. 

In [ ]:
lfs = [check, free, call_now, regex_I_am_free, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
LFAnalysis(L=L_test, lfs=lfs).lf_summary()

In [ ]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [ ]:
preds_train

In [ ]:
import numpy as np

labels, counts = np.unique(preds_train, return_counts=True)

for l, c in zip(labels, counts):
    print(f"LABEL: {l}, count: {c}")

In [ ]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)

In [ ]:
majority_acc = majority_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Majority voting accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Probabilistic model accuracy:':<25} {label_model_acc * 100:.1f}%")

Unfortunately, some data points will not receive any label. It is necessary to filter out these points before sending the labeling result for further processing.

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import preds_to_probs, probs_to_preds

preds_train, probs_train = label_model.predict(L=L_train, return_probs=True)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)
df_train.shape, df_train_filtered.shape

As you can see, we were able to quickly prepare labels for about 650 examples (recall that initially no example in the `df_train` set had labels).

The next step will use prepared labels as training data for the actual classifier. We will use simple [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), first pre-processing the input data. Since we are working with text, we will use the [word vector representation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) created based on 5-grams by `CountVectorizer`.

In [ ]:
from snorkel.utils import probs_to_preds
from sklearn.feature_extraction.text import CountVectorizer

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

vectorizer = CountVectorizer(ngram_range=(1, 5))

X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

In [ ]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver='lbfgs')
sklearn_model.fit(X=X_train, y=preds_train_filtered)

In [ ]:
print(f"Logistic regression accuracy: {sklearn_model.score(X=X_test, y=df_test.label) * 100:.1f}%")

As can be seen, the final model improved the score over the majority vote and the `LabelModel` model.

#### assignment

Complete the above calls with functions that you wrote yourself and check whether your functions improve the quality of the model.

## Transforming functions

The idea of a transforming function is to perform an atomic transformation of an instance. For data that is an image, typical transformations include cropping, rotating, and changing the color palette. For text data, you can replace words with synonyms, substitute named entities, cut random pieces of text, etc. In the following example we will find types of named entities occurring in the text, and then prepare a simple transformer that will randomly replace occurrences of the `PERSON` entity

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

for doc in nlp.pipe(df_train.text.sample(frac=0.05)):
    print(f"Entities: {[(e.text, e.label_) for e in doc.ents]}")

In [ ]:
person_entities = []

for doc in nlp.pipe(df_train.text):
    for e in doc.ents:
        if e.label_ == 'PERSON':
            person_entities.append(e.text)
        
person_entities[:10]

In [ ]:
from snorkel.augmentation import transformation_function
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

@transformation_function(pre=[spacy])
def random_person_ner(sms):
    person_ners = [e.text for e in sms.doc.ents]
    
    if person_ners:
        person_to_replace = np.random.choice(person_ners)
        person_to_add = np.random.choice(person_entities)
        sms.text = sms.text.replace(person_to_replace, person_to_add)
    return sms

Another example of transformation could be using WordNet to find synonyms for words. However, this requires downloading a corpus of data

In [ ]:
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")

In [ ]:
def get_synonym(word):
    
    synsets = wordnet.synsets(word)
    
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        
        return np.random.choice([w.replace("_", " ") for w in words])


In [ ]:
@transformation_function()
def replace_words_with_synonym(sms, num_replacements=5):

    words = sms.text.split()
    
    for _ in range(num_replacements):
        word_idx = np.random.choice(range(len(words)))
        synonym = get_synonym(words[word_idx])
        if synonym:
            words[word_idx] = synonym
        
    sms.text = ' '.join(words)
    return sms

Let us now compare the original text message content with the transformed versions.

In [ ]:
# source: https://github.com/snorkel-team/snorkel-tutorials/blob/master/spam/utils.py

from collections import OrderedDict

def preview_tfs(df, tfs):
    transformed_examples = []
    for f in tfs:
        for i, row in df.iterrows():
            transformed_or_none = f(row)
            # If TF returned a transformed example, record it in dict and move to next TF.
            if transformed_or_none is not None:
                transformed_examples.append(
                    OrderedDict(
                        {
                            "TF Name": f.name,
                            "Original Text": row.text,
                            "Transformed Text": transformed_or_none.text,
                        }
                    )
                )
                
    return pd.DataFrame(transformed_examples)


In [ ]:
tfs = [random_person_ner, replace_words_with_synonym]

preview_tfs(df_train.sample(frac=0.1), tfs)

Applying transforming functions requires some policy defining the order and number of transformations. In the example below, two transformation functions are drawn at random and this sequence of two functions is applied twice to each data point. As a result, we triple the size of the train set.

In [ ]:
from snorkel.augmentation import RandomPolicy, PandasTFApplier

random_policy = RandomPolicy(len(tfs), sequence_length=2, n_per_original=2, keep_original=True)

tf_applier = PandasTFApplier(tfs, random_policy)

df_train_sample = df_train.sample(frac=0.1)
df_train_augmented = tf_applier.apply(df_train_sample)

In [ ]:
df_train_sample.shape, df_train_augmented.shape

#### assignment

Modify the transforming function ``replace_words_with_synonym()`` so that you can restrict the replacement of words with synonyms only for specific parts of speech (e.g., replace only nouns or verbs).